In [1]:
import numpy as np
import pandas as pd
import random
import re
%matplotlib inline
from itertools import chain
import glob
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import datetime
import dateutil
from dateutil.relativedelta import relativedelta
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,log_loss
from sklearn.metrics import r2_score, balanced_accuracy_score, explained_variance_score


In [2]:
path = 'tennis_data/'

# df = pd.read_excel('tennis_data/2018.xlsx')

all_data_paths = glob.glob(os.path.join(path,'*.xlsx'))

frames = [pd.read_excel(i) for i in all_data_paths]

df = pd.concat(frames, sort=False)
df.reset_index(drop=True)

df['loser_surname']= df.Loser.str[:-3]
df['winner_surname']= df.Winner.str[:-3]
df['winner_initial'] = df.Winner.str[-2:-1]
df['loser_initial'] = df.Loser.str[-2:-1]


#
players = pd.read_csv('tennis_atp/atp_players.csv',encoding='iso-8859-1', names=['id', 'first_name', 'surname', 'hand', 'points','country'])

all_atp_data_paths = []
for i in range(2000,2018): all_atp_data_paths.append('tennis_atp/atp_matches_{year}.csv'.format(year=i))


frames = [pd.read_csv(i) for i in all_atp_data_paths]

# frames
atp = pd.concat(frames, sort=False)

atp.tourney_date = pd.to_datetime(atp.tourney_date, format='%Y%m%d')


len(df)

16278

In [3]:
df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets',
       'Lsets', 'Comment', 'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW',
       'PSL', 'MaxW', 'MaxL', 'AvgW', 'AvgL', 'SJW', 'SJL', 'loser_surname',
       'winner_surname', 'winner_initial', 'loser_initial'],
      dtype='object')

In [4]:
# create player 1 and 2 columns and sort winner/loser stats into p1/p2 stats

sort_list = [random.choice([True,False]) for i in range(len(df))]
inverse_list = [not i for i in sort_list]

df['P1'] = [winner if condition  else loser for winner, loser, condition in zip(df.Winner, df.Loser, sort_list) ]
df['P2'] = [winner if condition  else loser for winner, loser, condition in zip(df.Winner, df.Loser, inverse_list) ]


def sort_by_player(df, p1_stat, p2_stat, w_stat, l_stat, sort_list ):
    #creates player 1 and player 2 stats columsn from winner stat and loser stat using a boolean list for map
    inverse_list = [not i for i in sort_list]
    df[p1_stat] = [winner if condition  else loser for winner, loser, condition in zip(df[w_stat], df[l_stat], sort_list)]
    df[p2_stat] = [winner if condition  else loser for winner, loser, condition in zip(df[w_stat], df[l_stat], inverse_list )]

# df['P1Rank'] = [winner if condition  else loser for winner, loser, condition in zip(df.WRank, df.LRank, sort_list)]
# df['P2Rank'] = [winner if condition  else loser for winner, loser, condition in zip(df.WRank, df.LRank, inverse_list)]


columns_by_winner = ['WRank', 'LRank', 'WPts', 'LPts',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets',
       'Lsets', 'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW',
       'PSL', 'MaxW', 'MaxL', 'AvgW', 'AvgL']
columns_by_player = [re.sub('L', 'P2', j) for j in [re.sub('W', 'P1', i) for i in columns_by_winner]]


cw = [columns_by_winner[i:i+2] for i in range(0,len(columns_by_winner),2)]
cp = [columns_by_player[i:i+2] for i in range(0,len(columns_by_winner),2)]


for i in range(len(cw)): 
    cp[i].extend(cw[i])
    sort_by_player(df, *cp[i] ,sort_list)


In [5]:
df[['Winner', 'Loser', 'WRank', 'LRank','P1','P2', 'P1Rank', 'P2Rank']].head()

,Winner,Loser,WRank,LRank,P1,P2,P1Rank,P2Rank
0,Thompson J.,Ymer E.,79.0,160.0,Ymer E.,Thompson J.,160.0,79.0
1,Mahut N.,Robert S.,39.0,54.0,Robert S.,Mahut N.,54.0,39.0
2,Ferrer D.,Tomic B.,21.0,26.0,Ferrer D.,Tomic B.,21.0,26.0
3,Edmund K.,Escobedo E.,45.0,141.0,Edmund K.,Escobedo E.,45.0,141.0
4,Dimitrov G.,Johnson S.,17.0,33.0,Dimitrov G.,Johnson S.,17.0,33.0


In [6]:
#clusters taken from 310774506_Tennis_betting_Can_statistics_beat_bookmakers

def intervals(delta):
    if 0 < abs(delta) < 560:
        return 1*np.sign(delta)
    elif 560 <= abs(delta) < 920:
        return 2*np.sign(delta)
    elif 920 <= abs(delta) < 1460:
        return 3*np.sign(delta)
    elif 1460 <= abs(delta) < 2000:
        return 4*np.sign(delta)
    elif 2000 <= abs(delta):
        return 5*np.sign(delta)
    else:return None

df['PtsDelta'] = df.P1Pts - df.P2Pts

df['IntervalDelta'] = df.PtsDelta.apply(intervals)



In [7]:
df

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,P2BP1,P2BP2,PSP1,PSP2,MaxP1,MaxP2,AvgP1,AvgP2,PtsDelta,IntervalDelta
0,1,Brisbane,Brisbane International,2017-01-01,ATP250,Outdoor,Hard,1st Round,3,Thompson J.,...,3.50,1.30,3.73,1.31,3.90,1.33,3.50,1.29,-317.0,-1.0
1,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Mahut N.,...,2.50,1.53,2.52,1.58,2.55,1.59,2.43,1.54,-248.0,-1.0
2,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Ferrer D.,...,2.00,1.80,2.10,1.81,2.15,1.82,2.01,1.77,320.0,1.0
3,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Edmund K.,...,1.40,3.00,1.42,3.09,1.42,3.25,1.37,3.01,558.0,1.0
4,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Dimitrov G.,...,1.40,3.00,1.41,3.13,1.46,3.13,1.41,2.85,715.0,2.0
5,1,Brisbane,Brisbane International,2017-01-03,ATP250,Outdoor,Hard,1st Round,3,Donaldson J.,...,1.50,2.62,1.51,2.74,1.51,2.75,1.47,2.64,685.0,2.0
6,1,Brisbane,Brisbane International,2017-01-03,ATP250,Outdoor,Hard,1st Round,3,Groth S.,...,2.25,1.62,2.34,1.66,2.40,1.66,2.27,1.61,-378.0,-1.0
7,1,Brisbane,Brisbane International,2017-01-03,ATP250,Outdoor,Hard,1st Round,3,Pouille L.,...,1.91,1.91,1.93,1.95,1.93,2.00,1.83,1.94,571.0,2.0
8,1,Brisbane,Brisbane International,2017-01-03,ATP250,Outdoor,Hard,1st Round,3,Troicki V.,...,2.50,1.50,2.62,1.54,2.85,1.54,2.57,1.49,-781.0,-2.0
9,1,Brisbane,Brisbane International,2017-01-03,ATP250,Outdoor,Hard,1st Round,3,Schwartzman D.,...,2.20,1.67,2.20,1.74,2.30,1.74,2.18,1.66,-491.0,-1.0


In [8]:
# get birthday from age and and date 

def get_wbday(info):
    try: 
        age = info['winner_age']
        year = info['tourney_date']
        return year - relativedelta(years=int(age), days=int(((age-int(age))*365.24)))
    except ValueError as e: return np.nan

def get_lbday(info):
    try: 
        age = info['loser_age']
        year = info['tourney_date']
        return year - relativedelta(years=int(age), days=int(((age-int(age))*365.24)))
    except ValueError as e: return np.nan

atp['winner_birthday'] = atp.apply(get_wbday, axis=1)
atp['loser_birthday'] = atp.apply(get_lbday, axis=1)



In [9]:
atp1 = pd.concat([atp.winner_id,atp.winner_birthday], axis=1).rename(columns={'winner_id':'id', 'winner_birthday':'birthday'})

atp2 = pd.concat([atp.loser_id,atp.loser_birthday], axis=1).rename(columns={'loser_id':'id', 'loser_birthday':'birthday'})

players_only = atp1.append(atp2)

players_only.drop_duplicates(inplace=True, subset='id') #just player id and birthdayd extracted from atp match data


print(len(players), len(players_only))

53319 2152


In [10]:
def calculate_age(born, today):
    try:
        delta = relativedelta(today, born)
        return delta.years + delta.months/12
    except: return np.nan
#     return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

players_only.head()

,id,birthday
0,102179.0,1973-02-24
1,103602.0,1980-07-29
2,103387.0,1979-06-15
3,101733.0,1970-04-14
4,101727.0,1970-04-04


In [11]:
players_ = pd.merge(players_only, players, on='id') #addbirthday to atp players df. - all atp data

In [12]:
len(players_)
len(df)

16278

In [13]:
players_['initial'] = players_.first_name.str[0]

In [14]:
_wnames = players_[['surname','initial', 'id','birthday','country',]].rename(columns={'surname':'winner_surname','initial':'winner_initial','id':'winner_id', 'birthday': 'winner_birthday', 'country':'winner_country'})
_lnames = players_[['surname','initial', 'id','birthday','country',]].rename(columns={'surname':'loser_surname','initial':'loser_initial','id':'loser_id', 'birthday': 'loser_birthday', 'country':'loser_country'})

In [15]:
df = pd.merge(df, _wnames, how='inner', on=['winner_surname', 'winner_initial'], indicator='indicator_column')
df.winner_birthday.apply(pd.to_datetime)
df = pd.merge(df, _lnames, how='inner', on=['loser_surname', 'loser_initial'], indicator='indicator_column_loser_merge')

In [16]:
df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets',
       'Lsets', 'Comment', 'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW',
       'PSL', 'MaxW', 'MaxL', 'AvgW', 'AvgL', 'SJW', 'SJL', 'loser_surname',
       'winner_surname', 'winner_initial', 'loser_initial', 'P1', 'P2',
       'P1Rank', 'P2Rank', 'P1Pts', 'P2Pts', 'P11', 'P21', 'P12', 'P22', 'P13',
       'P23', 'P14', 'P24', 'P15', 'P25', 'P1sets', 'P2sets', 'B365P1',
       'B365P2', 'EXP1', 'EXP2', 'P2BP1', 'P2BP2', 'PSP1', 'PSP2', 'MaxP1',
       'MaxP2', 'AvgP1', 'AvgP2', 'PtsDelta', 'IntervalDelta', 'winner_id',
       'winner_birthday', 'winner_country', 'indicator_column', 'loser_id',
       'loser_birthday', 'loser_country', 'indicator_column_loser_merge'],
      dtype='object')

In [17]:
all_player_names = set(df.Winner) and set(df.Loser)
df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets',
       'Lsets', 'Comment', 'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW',
       'PSL', 'MaxW', 'MaxL', 'AvgW', 'AvgL', 'SJW', 'SJL', 'loser_surname',
       'winner_surname', 'winner_initial', 'loser_initial', 'P1', 'P2',
       'P1Rank', 'P2Rank', 'P1Pts', 'P2Pts', 'P11', 'P21', 'P12', 'P22', 'P13',
       'P23', 'P14', 'P24', 'P15', 'P25', 'P1sets', 'P2sets', 'B365P1',
       'B365P2', 'EXP1', 'EXP2', 'P2BP1', 'P2BP2', 'PSP1', 'PSP2', 'MaxP1',
       'MaxP2', 'AvgP1', 'AvgP2', 'PtsDelta', 'IntervalDelta', 'winner_id',
       'winner_birthday', 'winner_country', 'indicator_column', 'loser_id',
       'loser_birthday', 'loser_country', 'indicator_column_loser_merge'],
      dtype='object')

In [18]:
df['winner_age'] = df.apply(lambda row : calculate_age(row['winner_birthday'], row['Date']), axis=1)
df['loser_age'] = df.apply(lambda row : calculate_age(row['loser_birthday'], row['Date']), axis=1)
sort_by_player(df, 'P1Age', 'P2Age', 'winner_age', 'loser_age', sort_list)

In [19]:
df[['Winner', 'winner_age', 'Date', 'winner_birthday']].head()

,Winner,winner_age,Date,winner_birthday
0,Thompson J.,22.666667,2017-01-01,1994-04-21
1,Ferrer D.,33.000000,2015-04-23,1982-04-03
2,Dimitrov G.,22.083333,2013-07-08,1991-05-16
3,Schwartzman D.,23.000000,2015-08-31,1992-08-17
4,Verdasco F.,33.666667,2017-07-18,1983-11-16


In [20]:
players_[players_.id==102928.0]

,id,birthday,first_name,surname,hand,points,country,initial
500,102928.0,1977-02-03,Hee Seok,Chung,R,19770129.0,KOR,H


In [21]:
df['P1Result'] = pd.get_dummies(df.P1 == df.Winner, drop_first=True)

df.to_csv('data.csv', index=True, index_label='index')

In [22]:
train_on = [ 'PtsDelta', 'IntervalDelta' , 'P1Age', 'P2Age', 'P1Result']


# 'PtsDelta',IntervalDelta', 

In [23]:
model_data = df[train_on][df.Date < datetime.datetime(2016,12,31)].copy()
model_data = model_data.dropna()

In [24]:
X_train, X_test, y_train, y_test = train_test_split(model_data.drop('P1Result', axis=1),model_data['P1Result'], test_size=0.1)


In [25]:
logmodel = LogisticRegression(solver= 'liblinear', fit_intercept=True)
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [26]:
print(classification_report(y_test,predictions))
print(balanced_accuracy_score(y_test,predictions))
print(log_loss(y_test,logmodel.predict_proba(X_test)))

              precision    recall  f1-score   support

           0       0.67      0.65      0.66       439
           1       0.64      0.66      0.65       412

   micro avg       0.66      0.66      0.66       851
   macro avg       0.66      0.66      0.66       851
weighted avg       0.66      0.66      0.66       851

0.6557627662162462
0.6152878133761522


In [27]:
probs = logmodel.predict_proba(model_data.drop('P1Result', axis=1).iloc[0].values.reshape(1, -1))
np.flip(probs) # P1 Win / P2 Win probs

array([[0.15548887, 0.84451113]])

In [28]:
probs_odds = 1/probs
np.flip(probs)

array([[0.15548887, 0.84451113]])

In [ ]:
def expected_return(probs,odds):
    er_0 = probs[0,0]*(odds[0]-1) - probs[0,1]
    er_1 = probs[0,1]*(odds[1]-1) - probs[0,0]
    return (er_0, er_1)


odds = df.iloc[0][['B365P1','B365P2']]


#Kelly stake 

def kelly_ratio(er,odds):
    kr_0, kr_1 = er/(odds-1)
    return (max(0,kr_0), max(0,kr_1))

print(expected_return(np.flip(probs), odds ))
print(kelly_ratio(expected_return(np.flip(probs), odds ), odds))

In [ ]:
test_data = df[df.Date > datetime.datetime(2016,12,31)].copy()


In [ ]:
test_data['Probs'] = logmodel.predict_proba(test_data[[ 'PtsDelta', 'IntervalDelta' , 'P1Age', 'P2Age']].values)

In [ ]:
test_data[[ 'PtsDelta', 'IntervalDelta' , 'P1Age', 'P2Age']].values

In [ ]:
y_test.mean()

In [ ]:
2.718281828**(logmodel.intercept_)/(1+2.718281828**(logmodel.intercept_))